# ANN

Table of Contents
- [1. Preprocessing Data](#1-preprocessing-the-data)
- [2. Model for Calls ONLY](#2-model-for-calls-only)
  - [2.1 Stage 1 Model (Binary Classifier)](#21-stage-1-model-binary-classifier)
  - [2.2 Stage 2 Model (Price Prediction)](#22-stage-2-model-price-prediction)
  - [2.3 Testing model](#23-testing-model)
  - [2.4 Hyper param Tuning](#24-hyperparam-tuning)
    - [2.4.1 Hyperparam Tuning (Binary Classifier)](#241-hyperparam-tuning-binary-classifier)
    - [2.4.2 Hyperparam Tuning (Price Prediction)](#242-hyperparam-tuning-price-prediction)
- [3. Model for Puts ONLY](#3-model-for-puts-only)
  - [3.1 Stage 1 Model (Binary Classifier)](#31-stage-1-model-binary-classifier)
  - [3.2 Stage 2 Model (Price Prediction)](#32-stage-2-model-price-prediction)
  - [3.3 Testing model](#33-testing-model)
  - [3.4 Hyper param Tuning](#34-hyperparam-tuning)
    - [3.4.1 Hyperparam Tuning (Binary Classifier)](#341-hyperparam-tuning-binary-classifier)
    - [3.4.2 Hyperparam Tuning (Price Prediction)](#34-hyperparam-tuning-price-prediction)

<u>Stage 1: Binary Classification Model</u>

The first model is responsible for predicting if the option is “purchasable.” It classifies the input based on whether to proceed with price prediction or not.

- Input Layer
    - 15 neurons (one for each input feature).
- Hidden Layer
    - 1 Layer, 30 neurons (Leaky RelU)
- Output Layer
    - 1 neuron with a sigmoid activation function to produce a binary result:
        - Output = 1 if the option is “purchasable” (indicating "Buy the option").
        - Output = 0 if the option is “not purchasable” (indicating "Do not buy").

<u>Stage 2: Price Prediction Model</u>

Only if the classification model predicts the option as “purchasable” (1) does the data proceed to this model. This model predicts the price of the option.

- Input Layer
    - 15 neurons (same input features as the classification model).
- Hidden Layers (Price Prediction Layer)
    - First layer, 30 neurons (Leaky ReLU)
    - Second Layer, 15 neurons (Leaky ReLU)
- Output Layer
    - 1 neuron without activation (for regression) to predict the option price.

# 1. Preprocessing the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-07-26.csv')
df3 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-09-27.csv')
df2 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-08-30.csv')
df4 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-10-11.csv')
df5 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-10-18.csv')
df6 = pd.read_csv('data/SPX/UnderlyingOptionsIntervals_3600sec_calcs_oi_2024-10-25.csv')

In [3]:
df_train = pd.concat([
    df1[(df1['root']=='SPX')],
    df2[(df2['root']=='SPX')],
    df3[(df3['root']=='SPX')],
    df4[(df4['root']=='SPX')],
    df5[(df5['root']=='SPX')]
])
df_test = df6[df6['root'] == 'SPX'].copy()

In [4]:
# Transforming relevant columns to datetime
df_train['expiration'] = pd.to_datetime(df_train['expiration'])
df_train['quote_datetime'] = pd.to_datetime(df_train['quote_datetime'])

df_test['expiration'] = pd.to_datetime(df_test['expiration'])
df_test['quote_datetime'] = pd.to_datetime(df_test['quote_datetime'])

# Calculate time-to-expiration as the difference in days
df_train['time_to_expiration'] = (df_train['expiration'] - df_train['quote_datetime']).dt.days
df_test['time_to_expiration'] = (df_test['expiration'] - df_test['quote_datetime']).dt.days

In [5]:
# New column bid_ask_spread
df_train['bid_ask_spread'] = df_train['underlying_ask'] - df_train['underlying_bid']
df_test['bid_ask_spread'] = df_test['underlying_ask'] - df_test['underlying_bid']

In [6]:
# New column is_buy, shows whether an option was bought
df_train['is_buy'] = ((df_train['trade_volume'] > 0) & (df_train['close'] != 0)).astype(int)
df_test['is_buy'] = ((df_test['trade_volume'] > 0) & (df_test['close'] != 0)).astype(int)

In [7]:
df_train_calls = df_train[df_train['option_type']=='C']
df_test_calls = df_test[df_test['option_type']=='C']

df_train_puts = df_train[df_train['option_type']=='P']
df_test_puts = df_test[df_test['option_type']=='P']

In [8]:
df_test_calls.columns

Index(['underlying_symbol', 'quote_datetime', 'root', 'expiration', 'strike',
       'option_type', 'open', 'high', 'low', 'close', 'trade_volume',
       'bid_size', 'bid', 'ask_size', 'ask', 'underlying_bid',
       'underlying_ask', 'implied_underlying_price', 'active_underlying_price',
       'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho',
       'open_interest', 'time_to_expiration', 'bid_ask_spread', 'is_buy'],
      dtype='object')

In [9]:
# Dropping irrelevant columns
# root, underlying_symbol, option_type, implied_underlying_price, quote_datetime, expiration, bid_size, ask_size, underlying_bid, underlying_ask, open, high, low
columns_to_drop = ['root', 'option_type','underlying_symbol', 'implied_underlying_price', 'quote_datetime', 
                   'expiration', 'open', 'high', 'low', 'bid_size', 'ask_size', 
                   'underlying_bid', 'underlying_ask']

df_train_calls = df_train_calls.drop(columns = columns_to_drop, axis=1)
df_train_puts = df_train_puts.drop(columns = columns_to_drop, axis=1)
df_test_calls = df_test.drop(columns = columns_to_drop, axis=1)
df_test_puts = df_test.drop(columns = columns_to_drop, axis=1)

# 2. Model for Calls ONLY

In [10]:
# Scaling
from sklearn.preprocessing import StandardScaler

numerical_columns = ['strike', 'trade_volume', 'active_underlying_price', 
                     'implied_volatility', 'delta', 'gamma', 'theta', 
                     'vega', 'rho', 'open_interest', 'time_to_expiration', 'bid_ask_spread']
scaler = StandardScaler()
df_train_calls[numerical_columns] = scaler.fit_transform(df_train_calls[numerical_columns])
df_test_calls[numerical_columns] = scaler.transform(df_test_calls[numerical_columns])

## 2.1 Stage 1 Model (Binary Classifier)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam

# Define the binary classifier model
binary_classifier_calls = Sequential()

# Input layer with 14 neurons for 14 features
binary_classifier_calls.add(Input(shape=(14,)))

# Hidden layer with 28 neurons
binary_classifier_calls.add(Dense(units=14))
binary_classifier_calls.add(LeakyReLU(negative_slope=0.01))         # Leaky ReLU activation

# Output layer with 1 neuron and sigmoid activation for binary classification
binary_classifier_calls.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
binary_classifier_calls.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
binary_classifier_calls.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 14)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225 (900.00 B)

 Trainable params: 225 (900.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Separate features and target for training
X_train = df_train_calls.drop(columns=['close', 'is_buy'])  # Drop 'close' and 'is_buy' from features
y_train = df_train_calls['is_buy']  # Target for binary classification

In [13]:
# Training the binary classifier
history = binary_classifier_calls.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 250us/step - accuracy: 0.9459 - loss: 0.2080 - val_accuracy: 0.9503 - val_loss: 0.1612
Epoch 2/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 236us/step - accuracy: 0.9526 - loss: 0.1879 - val_accuracy: 0.9561 - val_loss: 0.1585
Epoch 3/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 251us/step - accuracy: 0.9560 - loss: 0.2003 - val_accuracy: 0.9497 - val_loss: 0.8570
Epoch 4/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - accuracy: 0.9591 - loss: 0.1875 - val_accuracy: 0.9573 - val_loss: 0.1604
Epoch 5/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - accuracy: 0.9605 - loss: 0.1817 - val_accuracy: 0.9593 - val_loss: 0.1383
Epoch 6/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - accuracy: 0.9607 - loss: 0.1714 - val_accuracy: 0.9625 - val_loss: 0.1261
Epoch 7/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 236us/step - accuracy: 0.9640 - loss: 0.1616 - val_accuracy: 0.9610 - val_loss: 0.1680
Epoch 8/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - accuracy: 0.9657 -

In [14]:
# Separate features and target for the price prediction model
X_train_calls = df_train_calls.drop(columns=['is_buy','close'])  # Features for price prediction
y_train_price_full = df_train_calls['close']  # Target price

# Generate the 'purchasable' predictions using the classifier
purchasable_mask = binary_classifier_calls.predict(X_train_calls) >= 1  # Prediction mask for purchasable options

# Filter rows for the price prediction model based on the 'purchasable' mask
X_train_price = X_train_calls[purchasable_mask.flatten()]  # Only rows classified as purchasable
y_train_price = y_train_price_full[purchasable_mask.flatten()]  # Corresponding target prices for purchasable options

4312/4312 ━━━━━━━━━━━━━━━━━━━━ 1s 165us/step


## 2.2 Stage 2 Model (Price Prediction)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam

# Define the price prediction model
price_prediction_model_calls = Sequential()

# Input layer with 15 neurons and first hidden layer with 30 neurons
price_prediction_model_calls.add(Dense(units=28, input_shape=(14,)))  # 15 input features
price_prediction_model_calls.add(LeakyReLU(negative_slope=0.01))               # Leaky ReLU activation

# Second hidden layer with 15 neurons and Leaky ReLU activation
price_prediction_model_calls.add(Dense(units=14))
price_prediction_model_calls.add(LeakyReLU(negative_slope=0.01))

# Output layer with 1 neuron (no activation for regression)
price_prediction_model_calls.add(Dense(units=1))

# Compile the model with Mean Squared Error (MSE) as the loss function for regression
price_prediction_model_calls.compile(optimizer=Adam(), loss='mse')

# Summary of the model architecture
price_prediction_model_calls.summary()


/Users/joelsng/Library/Python/3.12/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 28)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │           406 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the price prediction model
history = price_prediction_model_calls.fit(X_train_price, y_train_price, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 106339.9922 - val_loss: 144155.1250
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - loss: 118049.8984 - val_loss: 109661.1328
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - loss: 94801.9297 - val_loss: 85440.8594
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - loss: 32016.9355 - val_loss: 67557.2422
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - loss: 102656.1719 - val_loss: 39551.5938
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 38973.1641 - val_loss: 19325.4414
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - loss: 120131.9219 - val_loss: 8765.7852
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - loss: 22102.8750 - val_loss: 3213.1023
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 84533.4922 - val_loss: 869.8706
Epoch 10/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 29916.7754 - val_loss: 186.6564


## 2.3 Testing model

Testing is performed on a 6th day of trading not included in the training data

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [18]:
# Features and target for binary classification model
X_test_classification = df_test_calls.drop(columns=['close', 'is_buy'])  # Features for binary classification
y_test_classification = df_test_calls['is_buy']           # Binary target for classification

# Features and target for price prediction model
X_test_price = df_test_calls.drop(columns=['is_buy', 'close'])  # Features for price prediction
y_test_price_full = df_test_calls['close']                      # Target price for price prediction

In [19]:
y_pred_binary = (binary_classifier_calls.predict(X_test_classification).flatten() >= 0.5).astype(int)

# Calculate binary classifier metrics
binary_accuracy = accuracy_score(y_test_classification, y_pred_binary)
binary_precision = precision_score(y_test_classification, y_pred_binary)
binary_recall = recall_score(y_test_classification, y_pred_binary)
binary_f1 = f1_score(y_test_classification, y_pred_binary)

print("Binary Classifier Metrics:")
print(f"Accuracy: {binary_accuracy}")
print(f"Precision: {binary_precision}")
print(f"Recall: {binary_recall}")
print(f"F1-Score: {binary_f1}")


1744/1744 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step
Binary Classifier Metrics:
Accuracy: 0.9471184861300265
Precision: 0.9870863599677159
Recall: 0.29413179413179413
F1-Score: 0.4532147489345933


In [20]:
# Step 2: Filter Data for Stage 2 Price Prediction
purchasable_mask = (y_pred_binary == 1)  # Mask for options classified as "purchasable"

# Apply the mask to get features and target for price prediction
X_test_price_purchasable = X_test_price[purchasable_mask]
y_test_price = y_test_price_full[purchasable_mask]

In [21]:
# Step 3: Run Stage 2 Price Prediction
if len(X_test_price_purchasable) > 0:
    y_pred_price = price_prediction_model_calls.predict(X_test_price_purchasable)

    # Calculate regression metrics
    mse = mean_squared_error(y_test_price, y_pred_price)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_price, y_pred_price)
    r2 = r2_score(y_test_price, y_pred_price)

    print("\nPrice Prediction Model Metrics:")
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
else:
    print("\nNo 'purchasable' options found for price prediction in the 6th day test set.")

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step

Price Prediction Model Metrics:
Mean Squared Error: 51.81627007159826
Root Mean Squared Error: 7.198351899678027
Mean Absolute Error: 3.0421077135383263
R-squared: 0.997987193735199


## 2.4 Hyperparam Tuning

### 2.4.1 Hyperparam Tuning (Binary Classifier)

In [22]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV 

In [23]:
# Define the model-building function for Random Search
def build_binary_classifier_model(units=28, learning_rate=1e-3):
    model = Sequential()
    
    # Define the input layer separately using Input()
    model.add(Input(shape=(14,)))
    
    # Hidden layer with Leaky ReLU activation (using `negative_slope` instead of `alpha`)
    model.add(Dense(units=units))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Output layer with 1 neuron and sigmoid activation for binary classification
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model with specified learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Wrap the model using KerasClassifier, passing the function as build_fn
binary_classifier = KerasClassifier(model=build_binary_classifier_model, verbose=0)


In [24]:
param_grid = {
    'model__units': [16, 32, 64],            # Number of neurons in the hidden layer
    'model__learning_rate': [1e-2, 1e-3, 1e-4],  # Learning rates to try
    'batch_size': [16, 32, 64],              # Batch sizes to try
    'epochs': [10, 20]                       # Number of epochs to train
}

In [25]:
# Initialize RandomizedSearchCV with the desired number of iterations
random_search = RandomizedSearchCV(estimator=binary_classifier, param_distributions=param_grid, 
                                   scoring='accuracy', cv=3, n_iter=10, random_state=42)

# Fit RandomizedSearchCV
X_train = df_train_calls.drop(columns=['close', 'is_buy'])  # Drop 'close' and 'is_buy' from features
y_train = df_train_calls['is_buy']  # Target for binary classification
random_search_result = random_search.fit(X_train, y_train)

# Get the best hyperparameters and score
print(f"Best Hyperparameters: {random_search_result.best_params_}")
print(f"Best Score: {random_search_result.best_score_}")


Best Hyperparameters: {'model__units': 16, 'model__learning_rate': 0.001, 'epochs': 20, 'batch_size': 16}
Best Score: 0.9752674869351288


### 2.4.2 Hyperparam Tuning (Price Prediction)

In [26]:
from scikeras.wrappers import KerasRegressor

In [27]:
# Define the model-building function
def build_price_prediction_model(units1=28, units2=14, learning_rate=1e-3):
    model = Sequential()
    
    # Input and first hidden layer
    model.add(Input(shape=(14,)))
    model.add(Dense(units=units1))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Second hidden layer
    model.add(Dense(units=units2))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Output layer
    model.add(Dense(units=1))  # No activation function for regression
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [28]:
# Wrap the model with KerasRegressor
price_prediction_model = KerasRegressor(model=build_price_prediction_model, verbose=0)

In [29]:
param_grid = {
    'model__units1': [16, 28, 32],               # Number of neurons in the first hidden layer
    'model__units2': [8, 14, 16],                # Number of neurons in the second hidden layer
    'model__learning_rate': [1e-2, 1e-3, 1e-4],  # Learning rates to try
    'batch_size': [16, 32, 64],                  # Batch sizes to try
    'epochs': [10, 20]                           # Number of epochs
}

In [30]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=price_prediction_model, param_distributions=param_grid, 
                                   scoring='neg_mean_squared_error', n_iter=10, cv=3, random_state=42)

# Fit RandomizedSearchCV
X_train = df_train_calls.drop(columns=['close', 'is_buy'])  # Features for training
y_train = df_train_calls['close']  # Target prices for training
random_search_result = random_search.fit(X_train, y_train)

# Get the best hyperparameters and score
print(f"Best Hyperparameters: {random_search_result.best_params_}")
print(f"Best Score (Negative MSE): {random_search_result.best_score_}")


Best Hyperparameters: {'model__units2': 14, 'model__units1': 28, 'model__learning_rate': 0.001, 'epochs': 20, 'batch_size': 32}
Best Score (Negative MSE): -11676.835853522614


# 3. Model for Puts ONLY

In [31]:
# Scaling
numerical_columns = ['strike', 'trade_volume', 'active_underlying_price', 
                     'implied_volatility', 'delta', 'gamma', 'theta', 
                     'vega', 'rho', 'open_interest', 'time_to_expiration', 'bid_ask_spread']
scaler = StandardScaler()
df_train_puts[numerical_columns] = scaler.fit_transform(df_train_puts[numerical_columns])
df_test_puts[numerical_columns] = scaler.transform(df_test_puts[numerical_columns])

## 3.1 Stage 1 Model (Binary Classifier)

In [32]:
# Define the binary classifier model
binary_classifier_puts = Sequential()

# Input layer with 14 neurons for 14 features
binary_classifier_puts.add(Input(shape=(14,)))

# Hidden layer with 28 neurons
binary_classifier_puts.add(Dense(units=14))
binary_classifier_puts.add(LeakyReLU(negative_slope=0.01))         # Leaky ReLU activation

# Output layer with 1 neuron and sigmoid activation for binary classification
binary_classifier_puts.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
binary_classifier_puts.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
binary_classifier_puts.summary()


Model: "sequential_64"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_160 (Dense)               │ (None, 14)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_96 (LeakyReLU)      │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_161 (Dense)               │ (None, 1)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225 (900.00 B)

 Trainable params: 225 (900.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Separate features and target for training
X_train = df_train_puts.drop(columns=['close', 'is_buy'])  # Drop 'close' and 'is_buy' from features
y_train = df_train_puts['is_buy']  # Target for binary classification

In [34]:
# Training the binary classifier
history = binary_classifier_puts.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 265us/step - accuracy: 0.8982 - loss: 0.3079 - val_accuracy: 0.9011 - val_loss: 0.2693
Epoch 2/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 254us/step - accuracy: 0.9221 - loss: 0.2478 - val_accuracy: 0.9076 - val_loss: 0.3721
Epoch 3/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step - accuracy: 0.9312 - loss: 0.2216 - val_accuracy: 0.9169 - val_loss: 0.2559
Epoch 4/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - accuracy: 0.9367 - loss: 0.2099 - val_accuracy: 0.9250 - val_loss: 0.2027
Epoch 5/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 245us/step - accuracy: 0.9406 - loss: 0.1999 - val_accuracy: 0.9335 - val_loss: 0.2119
Epoch 6/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 241us/step - accuracy: 0.9454 - loss: 0.1837 - val_accuracy: 0.9219 - val_loss: 0.4096
Epoch 7/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step - accuracy: 0.9448 - loss: 0.1984 - val_accuracy: 0.9355 - val_loss: 0.1936
Epoch 8/10
3449/3449 ━━━━━━━━━━━━━━━━━━━━ 1s 241us/step - accuracy: 0.9483 -

In [35]:
# Separate features and target for the price prediction model
X_train_puts = df_train_puts.drop(columns=['is_buy','close'])  # Features for price prediction
y_train_price_full = df_train_puts['close']  # Target price

# Generate the 'purchasable' predictions using the classifier
purchasable_mask = binary_classifier_puts.predict(X_train_puts) >= 1  # Prediction mask for purchasable options

# Filter rows for the price prediction model based on the 'purchasable' mask
X_train_price = X_train_puts[purchasable_mask.flatten()]  # Only rows classified as purchasable
y_train_price = y_train_price_full[purchasable_mask.flatten()]  # Corresponding target prices for purchasable options

4312/4312 ━━━━━━━━━━━━━━━━━━━━ 1s 164us/step


## 3.2 Stage 2 Model (Price Prediction)

In [36]:
# Define the price prediction model
price_prediction_model_puts = Sequential()

# Input layer with 15 neurons and first hidden layer with 30 neurons
price_prediction_model_puts.add(Dense(units=28, input_shape=(14,)))  # 15 input features
price_prediction_model_puts.add(LeakyReLU(negative_slope=0.01))               # Leaky ReLU activation

# Second hidden layer with 15 neurons and Leaky ReLU activation
price_prediction_model_puts.add(Dense(units=14))
price_prediction_model_puts.add(LeakyReLU(negative_slope=0.01))

# Output layer with 1 neuron (no activation for regression)
price_prediction_model_puts.add(Dense(units=1))

# Compile the model with Mean Squared Error (MSE) as the loss function for regression
price_prediction_model_puts.compile(optimizer=Adam(), loss='mse')

# Summary of the model architecture
price_prediction_model_puts.summary()


/Users/joelsng/Library/Python/3.12/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_65"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_162 (Dense)               │ (None, 28)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_97 (LeakyReLU)      │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_163 (Dense)               │ (None, 14)             │           406 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_98 (LeakyReLU)      │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_164 (Dense)               │ (None, 1)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Train the price prediction model
history = price_prediction_model_puts.fit(X_train_price, y_train_price, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss: 4348.3760 - val_loss: 12.3963
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 8.9014 - val_loss: 3.2556
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - loss: 5.5355 - val_loss: 2.1421
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step - loss: 4.3139 - val_loss: 1.5862
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step - loss: 3.8316 - val_loss: 1.6272
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 335us/step - loss: 3.2074 - val_loss: 1.4474
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: 3.3757 - val_loss: 2.1728
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - loss: 3.1734 - val_loss: 1.0625
Epoch 9/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - loss: 3.3143 - val_loss: 0.9708
Epoch 10/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - loss: 3.3515 - val_loss: 1.0063


## 3.3 Testing model

Testing is performed on a 6th day of trading not included in the training data

In [38]:
# Features and target for binary classification model
X_test_classification = df_test_puts.drop(columns=['close', 'is_buy'])  # Features for binary classification
y_test_classification = df_test_puts['is_buy']           # Binary target for classification

# Features and target for price prediction model
X_test_price = df_test_puts.drop(columns=['is_buy', 'close'])  # Features for price prediction
y_test_price_full = df_test_puts['close']                      # Target price for price prediction

In [39]:
y_pred_binary = (binary_classifier_puts.predict(X_test_classification).flatten() >= 0.5).astype(int)

# Calculate binary classifier metrics
binary_accuracy = accuracy_score(y_test_classification, y_pred_binary)
binary_precision = precision_score(y_test_classification, y_pred_binary)
binary_recall = recall_score(y_test_classification, y_pred_binary)
binary_f1 = f1_score(y_test_classification, y_pred_binary)

print("Binary Classifier Metrics:")
print(f"Accuracy: {binary_accuracy}")
print(f"Precision: {binary_precision}")
print(f"Recall: {binary_recall}")
print(f"F1-Score: {binary_f1}")


1744/1744 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
Binary Classifier Metrics:
Accuracy: 0.9592502329582109
Precision: 0.8046571798188874
Recall: 0.5983645983645983
F1-Score: 0.6863448275862069


In [40]:
# Step 2: Filter Data for Stage 2 Price Prediction
purchasable_mask = (y_pred_binary == 1)  # Mask for options classified as "purchasable"

# Apply the mask to get features and target for price prediction
X_test_price_purchasable = X_test_price[purchasable_mask]
y_test_price = y_test_price_full[purchasable_mask]

In [41]:
# Step 3: Run Stage 2 Price Prediction
if len(X_test_price_purchasable) > 0:
    y_pred_price = price_prediction_model_puts.predict(X_test_price_purchasable)

    # Calculate regression metrics
    mse = mean_squared_error(y_test_price, y_pred_price)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_price, y_pred_price)
    r2 = r2_score(y_test_price, y_pred_price)

    print("\nPrice Prediction Model Metrics:")
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
else:
    print("\nNo 'purchasable' options found for price prediction in the 6th day test set.")

97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step

Price Prediction Model Metrics:
Mean Squared Error: 8255.584990701605
Root Mean Squared Error: 90.86024978339871
Mean Absolute Error: 35.23507897853057
R-squared: 0.7967300392128884


## 3.4 Hyperparam Tuning

### 3.4.1 Hyperparam Tuning (Binary Classifier)

In [42]:
# Define the model-building function for Random Search
def build_binary_classifier_model(units=28, learning_rate=1e-3):
    model = Sequential()
    
    # Define the input layer separately using Input()
    model.add(Input(shape=(14,)))
    
    # Hidden layer with Leaky ReLU activation (using `negative_slope` instead of `alpha`)
    model.add(Dense(units=units))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Output layer with 1 neuron and sigmoid activation for binary classification
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model with specified learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Wrap the model using KerasClassifier, passing the function as build_fn
binary_classifier = KerasClassifier(model=build_binary_classifier_model, verbose=0)


In [43]:
param_grid = {
    'model__units': [16, 32, 64],            # Number of neurons in the hidden layer
    'model__learning_rate': [1e-2, 1e-3, 1e-4],  # Learning rates to try
    'batch_size': [16, 32, 64],              # Batch sizes to try
    'epochs': [10, 20]                       # Number of epochs to train
}

In [44]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize RandomizedSearchCV with the desired number of iterations
random_search = RandomizedSearchCV(estimator=binary_classifier, param_distributions=param_grid, 
                                   scoring='accuracy', cv=3, n_iter=10, random_state=42)

# Fit RandomizedSearchCV
X_train = df_train_puts.drop(columns=['close', 'is_buy'])  # Drop 'close' and 'is_buy' from features
y_train = df_train_puts['is_buy']  # Target for binary classification
random_search_result = random_search.fit(X_train, y_train)

# Get the best hyperparameters and score
print(f"Best Hyperparameters: {random_search_result.best_params_}")
print(f"Best Score: {random_search_result.best_score_}")


Best Hyperparameters: {'model__units': 16, 'model__learning_rate': 0.001, 'epochs': 20, 'batch_size': 16}
Best Score: 0.95968283098447


### 3.4.2 Hyperparam Tuning (Price Prediction)

In [45]:
# Define the model-building function
def build_price_prediction_model(units1=28, units2=14, learning_rate=1e-3):
    model = Sequential()
    
    # Input and first hidden layer
    model.add(Input(shape=(14,)))
    model.add(Dense(units=units1))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Second hidden layer
    model.add(Dense(units=units2))
    model.add(LeakyReLU(negative_slope=0.01))
    
    # Output layer
    model.add(Dense(units=1))  # No activation function for regression
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [46]:
# Wrap the model with KerasRegressor
price_prediction_model = KerasRegressor(model=build_price_prediction_model, verbose=0)

In [47]:
param_grid = {
    'model__units1': [16, 28, 32],               # Number of neurons in the first hidden layer
    'model__units2': [8, 14, 16],                # Number of neurons in the second hidden layer
    'model__learning_rate': [1e-2, 1e-3, 1e-4],  # Learning rates to try
    'batch_size': [16, 32, 64],                  # Batch sizes to try
    'epochs': [10, 20]                           # Number of epochs
}

In [48]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=price_prediction_model, param_distributions=param_grid, 
                                   scoring='neg_mean_squared_error', n_iter=10, cv=3, random_state=42)

# Fit RandomizedSearchCV
X_train = df_train_puts.drop(columns=['close', 'is_buy'])  # Features for training
y_train = df_train_puts['close']  # Target prices for training
random_search_result = random_search.fit(X_train, y_train)


# Get the best hyperparameters and score
print(f"Best Hyperparameters: {random_search_result.best_params_}")
print(f"Best Score (Negative MSE): {random_search_result.best_score_}")


Best Hyperparameters: {'model__units2': 14, 'model__units1': 28, 'model__learning_rate': 0.001, 'epochs': 20, 'batch_size': 32}
Best Score (Negative MSE): -2959.088906086525
